In [1]:
import numpy as np
import pandas as pd

from copy import deepcopy
from itertools import product

## Shanten Framework Calculations

Pre-compute statistics for hands by suit for shanten and other combinatoric shenanigans.
Design follows the guidelines described in [this blog post](http://blog.ezyang.com/2014/04/calculating-shanten-in-mahjong/).
In particular, tiles can be filtered from 9-element vectors instead of being drawn procedurally, and hand states can be evaluated systematically from related previous states.

### Data Attributes

For numeric suits:
- `tile_int`(index) - standard numeric representation of tiles
- `tile_vector` - 9-element vector representation of tiles
- `n_tiles` - number of tiles in the tile group
- `n_sets` - number of sets in the tile group
- `n_triplets` - number of triplets that can be interpreted from sets
- `n_sequences` - number of sequences that can be interpreted from sets (note that triplets and sequences can overlap)
- `n_blocks` - number of incomplete two or three tile groups
- `n_pairs` - number of pairs in the tile group
- `max_pairs` - maximum number of interpretable pairs in the tile group
- `n_koritsu` - number of unconnected tiles (includes quad = triplet + isolated interpretation)
- `n_terminals` - number of unique terminals
- `n_ways` - number of ways to draw the tiles from a full set

For honors:
- `tile_int` (index) - standard numeric representation of tiles
- `tile_vector` - 7-element vector representation of tiles
- `n_tiles` - number of tiles in the tile group
- `n_triplets` - number of triplets in the tile group
- `n_pairs` - number of pairs in the tile group
- `n_koritsu` - number of unconnected tiles (includes quad = triplet + isolated interpretation)
- `n_terminals` - number of unique terminals
- `n_ways` - number of ways to draw the tiles from a full set

In [2]:
def vector_to_string(t_vector):
    t_str = ''
    for cnt in t_vector:
        t_str += str(cnt)
    if t_str:
        return t_str
    else:
        return '0'

def vector_to_int(t_vector):
    t_int = ''
    for i, cnt in zip(np.arange(1,len(t_vector)+1),t_vector):
        t_int += cnt * str(i)
    if t_int:
        return int(t_int)
    else:
        return 0

def vector_to_ways(t_vector):
    ways_map = [1, 4, 6, 4, 1]
    ways = 1
    for cnt in t_vector:
        ways *= ways_map[cnt]
    return ways

### Honor Calculations

In [3]:
# generate all possible tile groups, sorted by amount
generator = product([0, 1, 2, 3, 4],repeat=7)

valid_sets = []

for tile_group in generator:
    n_tiles = sum(tile_group)
    if n_tiles <= 14:
        valid_sets.append(tile_group)

In [4]:
# create and populate a DataFrame with honors statistics
col_names = ['tile_vector', 'n_tiles', 'n_triplets', 'n_pairs',
             'n_koritsu', 'n_terminals', 'n_ways',]
honors_df = pd.DataFrame(columns=col_names, dtype=int)

for tiles in valid_sets:
    tile_array = np.array(tiles)
    tile_idx = vector_to_int(tile_array)
    new_row = {'tile_vector':  vector_to_string(tile_array),
               'n_tiles':      tile_array.sum(),
               'n_triplets':  (tile_array >= 3).sum(),
               'n_pairs':     (tile_array == 2).sum(),
               'n_koritsu':   (tile_array == 1).sum() + (tile_array == 4).sum(),
               'n_terminals': (tile_array >  0).sum(),
               'n_ways':       vector_to_ways(tiles),}
    new_row_df = pd.DataFrame(new_row, index=[tile_idx])
    honors_df  = pd.concat([honors_df, new_row_df])

In [5]:
# sort hands by tile count and values, then export to csv
honors_df.sort_index(inplace=True)
honors_df.to_csv('./shanten_jihai.csv',index_label='tile_int')

### Numeric Tile Calculations

In [6]:
# define lists of taatsu and sequences for processing of small-number tile groups
terminals = np.array([1,0,0,0,0,0,0,0,1])

taatsu = np.array([[1,1,0,0,0,0,0,0,0], [1,0,1,0,0,0,0,0,0], [0,1,1,0,0,0,0,0,0],
                   [0,1,0,1,0,0,0,0,0], [0,0,1,1,0,0,0,0,0], [0,0,1,0,1,0,0,0,0],
                   [0,0,0,1,1,0,0,0,0], [0,0,0,1,0,1,0,0,0], [0,0,0,0,1,1,0,0,0],
                   [0,0,0,0,1,0,1,0,0], [0,0,0,0,0,1,1,0,0], [0,0,0,0,0,1,0,1,0],
                   [0,0,0,0,0,0,1,1,0], [0,0,0,0,0,0,1,0,1], [0,0,0,0,0,0,0,1,1],])

ryankan = np.array([[1,0,1,0,1,0,0,0,0], [0,1,0,1,0,1,0,0,0], [0,0,1,0,1,0,1,0,0],
                    [0,0,0,1,0,1,0,1,0], [0,0,0,0,1,0,1,0,1],])

sequences = np.array([[1,1,1,0,0,0,0,0,0], [0,1,1,1,0,0,0,0,0], [0,0,1,1,1,0,0,0,0],
                      [0,0,0,1,1,1,0,0,0], [0,0,0,0,1,1,1,0,0], [0,0,0,0,0,1,1,1,0],
                      [0,0,0,0,0,0,1,1,1],])

In [7]:
# generate all possible tile sets, sorted by amount
# larger tile sets will rely on smaller tile sets for statistic computations
generator = product([0, 1, 2, 3, 4],repeat=9)

valid_sets = [[] for i in range(15)]

for tile_group in generator:
    n_tiles = sum(tile_group)
    if n_tiles <= 14:
        valid_sets[n_tiles].append(tile_group)

In [8]:
# create and populate a DataFrame with numeric tile statistics
col_names = ['tile_vector', 'n_tiles', 'n_sets', 'n_triplets', 'n_sequences',
             'n_blocks', 'n_pairs', 'max_pairs', 'n_koritsu', 'n_terminals', 'n_ways',]
suited_df = pd.DataFrame(columns=col_names, dtype=int)

In [9]:
# set up DataFrame with calculations for up to three tiles
for n_tiles in np.arange(0,4):
    for tiles in valid_sets[n_tiles]:
        tile_array = np.array(tiles)
        tile_idx = vector_to_int(tile_array)

        n_triplets  = (tile_array >= 3).sum()
        n_sequences = (np.count_nonzero(tile_array * sequences,axis=1) == 3).sum()
        has_ryankan  = (np.count_nonzero(tile_array * ryankan,axis=1) == 3).sum()
    
        if n_triplets or n_sequences:
            # all tiles are in a set, so no extra blocks or lone tiles
            n_pairs    = 0
            has_taatsu = 0
            n_koritsu  = 0
        elif has_ryankan:
            n_pairs    = 0
            has_taatsu = 1
            n_koritsu  = 0
        else:
            n_pairs   = (tile_array == 2).sum()
            # count at most one taatsu for complex blocks (e.g. 124)
            has_taatsu = min((np.count_nonzero(tile_array * taatsu,axis=1) == 2).sum(),1)

            if n_pairs + has_taatsu == 0:
                n_koritsu = n_tiles
            else:
                # account for overlaps in lone tile calculation
                n_koritsu = n_tiles - (1 + n_pairs + has_taatsu)

        new_row = {'tile_vector': vector_to_string(tile_array),
                   'n_tiles':     n_tiles,
                   'n_sets':      max(n_triplets, n_sequences),
                   'n_triplets':  n_triplets,
                   'n_sequences': n_sequences,
                   'n_blocks':    min(max(n_pairs, has_taatsu), 1),
                   'n_pairs':     n_pairs,
                   'max_pairs':   n_pairs,
                   'n_koritsu':   n_koritsu,
                   'n_terminals': np.count_nonzero(tile_array * terminals),
                   'n_ways':      vector_to_ways(tiles),}
        new_row_df = pd.DataFrame(new_row, index=[tile_idx])
        suited_df = pd.concat([suited_df, new_row_df])

In [10]:
# function for splitting tile groups into subgroups
def add_tile_split(new_row_df, tile_array, split_group):
    split_int = vector_to_int(split_group)
    remainder_int = vector_to_int(tile_array - split_group)
    temp_row = suited_df.loc[split_int] + suited_df.loc[remainder_int]
    return pd.concat([new_row_df, temp_row.to_frame().T])

for n_tiles in np.arange(4,15):
    for tiles in valid_sets[n_tiles]:
        tile_array = np.array(tiles)
        tile_idx = vector_to_int(tile_array)

        # dynamic programming approach: take out a set or block, then add components
        triplet_idx  = np.flatnonzero(tile_array >= 3)
        sequence_idx = np.flatnonzero(np.count_nonzero(tile_array * sequences,axis=1) == 3)

        new_row_df = pd.DataFrame(columns=col_names)
        if triplet_idx.size + sequence_idx.size > 0:
            for i in triplet_idx:
                triplet = np.zeros(9, dtype=int)
                triplet[i] += 3
                new_row_df = add_tile_split(new_row_df, tile_array, triplet)
            for i in sequence_idx:
                new_row_df = add_tile_split(new_row_df, tile_array, sequences[i])
        else:
            pair_idx    = np.flatnonzero(tile_array == 2)
            taatsu_idx  = np.flatnonzero(np.count_nonzero(tile_array * taatsu,axis=1) == 2)
            ryankan_idx = np.flatnonzero(np.count_nonzero(tile_array * ryankan,axis=1) == 3)
            
            for i in pair_idx:
                pair = np.zeros(9, dtype=int)
                pair[i] += 2
                new_row_df = add_tile_split(new_row_df, tile_array, pair)
            for i in taatsu_idx:
                new_row_df = add_tile_split(new_row_df, tile_array, taatsu[i])
                taatsu_elements = np.flatnonzero(taatsu[i])
                # checking for complex (3-tile) taatsu
                for j in taatsu_elements:
                    if tile_array[j] == 2:
                        complex_taatsu = deepcopy(taatsu[i])
                        complex_taatsu[j] += 1
                        new_row_df = add_tile_split(new_row_df, tile_array, complex_taatsu)
            for i in ryankan_idx:
                new_row_df = add_tile_split(new_row_df, tile_array, ryankan[i])

        # candidate statistics should maximize sets, then minimize koritsu
        max_sets = new_row_df['n_sets'] == new_row_df['n_sets'].max()
        new_row_df = new_row_df[max_sets]
        min_koritsu = new_row_df['n_koritsu'] == new_row_df['n_koritsu'].min()
        new_row_df = new_row_df[min_koritsu]

        # add new row to the output DataFrame
        new_row = {'tile_vector': vector_to_string(tile_array),
                   'n_tiles':     n_tiles,
                   'n_sets':      new_row_df['n_sets'].max(),
                   'n_triplets':  new_row_df['n_triplets'].max(),
                   'n_sequences': new_row_df['n_sequences'].max(),
                   'n_blocks':    new_row_df['n_blocks'].max(),
                   'n_pairs':     new_row_df['n_pairs'].max(),
                   'max_pairs':  (tile_array == 2).sum(),
                   'n_koritsu':   new_row_df['n_koritsu'].min(),
                   'n_terminals': np.count_nonzero(tile_array * terminals),
                   'n_ways':      vector_to_ways(tiles),}
        new_row_df = pd.DataFrame(new_row, index=[tile_idx])
        suited_df  = pd.concat([suited_df, new_row_df])

In [11]:
# sort hands by tile count and values, then export to csv
suited_df.sort_index(inplace=True)
suited_df.to_csv('./shanten_suuhai.csv',index_label='tile_int')

### Debugging and Testing
- 84 Tiles: Bamboo, Circles, and Dragons
- Hand: 3x Sets + 1x Pair

In [12]:
# all possible hands
no_winds = honors_df['tile_vector'].apply(lambda x: x[:4] == '0000')
dragons_df = honors_df[no_winds]
dragons_count = dragons_df.groupby('n_tiles').sum(numeric_only=True)['n_ways']
suited_count = suited_df.groupby('n_tiles').sum(numeric_only=True)['n_ways']

n_hands = 0
for drg in range(12):
    for sou in range(12-drg):
        pin = 11 - drg - sou
        n_hands += dragons_count[drg] * suited_count[sou] * suited_count[pin]
print(n_hands)

18574174153080


In [13]:
dragons_complete = dragons_df.query('(3 * n_triplets + 2 * n_pairs == n_tiles) & (n_pairs <= 1)')
dragons_complete_ways = dragons_complete.groupby(['n_tiles', 'n_triplets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()
dragons_complete_ways

,n_tiles,n_triplets,n_pairs,n_ways
0,0,0,0,1
1,2,0,1,18
2,3,1,0,12
3,5,1,1,144
4,6,2,0,48
5,8,2,1,288
6,9,3,0,64


In [14]:
suited_complete = suited_df.query('(3 * n_sets + 2 * n_pairs == n_tiles) & (n_pairs <= 1)')
suited_complete_ways = suited_complete.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()
suited_complete_ways

,n_tiles,n_sets,n_pairs,n_ways
0,0,0,0,1
1,2,0,1,54
2,3,1,0,484
3,5,1,1,19200
4,6,2,0,65272
5,8,2,1,1748756
6,9,3,0,2742868
7,11,3,1,47037380
8,12,4,0,40399783
9,14,4,1,440593684


In [15]:
# all winning hands
n_hands = 0
for drg in range(7):
    for sou in range(8-drg):
        pin = 7 - drg - sou
        if dragons_complete_ways.loc[drg,'n_tiles'] + suited_complete_ways.loc[sou,'n_tiles'] + suited_complete_ways.loc[pin,'n_tiles'] == 11:
            n_hands += dragons_complete_ways.loc[drg,'n_ways'] * suited_complete_ways.loc[sou,'n_ways'] * suited_complete_ways.loc[pin,'n_ways']
print(n_hands)

6232346696


In [16]:
# all simples
suited_simples = suited_df.query('(3 * n_sets + 2 * n_pairs == n_tiles) & (n_pairs <= 1) & (n_terminals == 0)')
suited_simples_ways = suited_simples.groupby(['n_tiles', 'n_sets', 'n_pairs']).sum(numeric_only=True)['n_ways'].reset_index()

n_hands = 0
for sou in range(8):
    pin = 7 - sou
    n_hands += suited_simples_ways.loc[sou,'n_ways'] * suited_simples_ways.loc[pin,'n_ways']
print(n_hands)

835960840
